In [309]:
import numpy as np  

In [ ]:
def hypothesis(theta, X, logistic=False):
    if not logistic:
        
    else:
        

In [298]:
def gradient_descent(X, Y, learning_rate=0.01, max_iters=10000, regularization=0.1):
    """Returns the optimum theta using gradient descent.
       Remember that you may need to normalize your data.
       We use regu"""
    
    # Number of training examples.
    m = X.shape[0]

    # When true we stop calculating the optimum value of theta.
    done = False
    
    # We declare convergence if J(theta) decreases by less than below.
    convergence = 10 ** -7
    
    # Assign theta to be a value of 0s initially.
    theta = np.array([0 for x in range(X.shape[1] + 1)], dtype='float')
    theta = np.reshape(theta, (-1,1))
    # There should be as many thetas as features.
    
    
    # Need to add ones to X
    ones = np.ones((1,m))
    
    # Append the columns of ones to x
    X = np.insert(X, 0, ones, axis=1)        
    
    for i in range(max_iters):
        
        # Output if the maximum number of iterations has been reached.
        if i == max_iters - 1:
            print("The maximum number of iterations has been reached!")
        
        # Need to copy theta
        prev_theta = theta                           
                              
        # Update theta using the grdient descent formula.
        # Remember this requires simultaneous updating.
        prediction = np.matmul(X, theta)
        
        difference = prediction - Y       
        
        # Calculate the sum.
        sigma = np.dot((prediction - Y)[:,0], X).reshape((-1,1))
        
        theta_0 = theta[0:1, :] - ((learning_rate / m) * sigma[0:1, :])
        theta_rest = theta[1:, :] * \
                        (1 - (learning_rate * regularization / m)) - ((learning_rate / m) * sigma[1:,:])

                                                                                
        theta =  np.insert(theta_rest, 0, theta_0, axis=0)
        
        
        #print("Theta: " + str(theta), "Prev Theta: " + str(prev_theta))
        
        if abs((theta - prev_theta).max(axis=0)) < convergence:
            print("Converged!")
            break            
        
    return theta
          
X = [[1,6], [4,4], [3,3], [9,3], [22,3], [133,3], [33,3], [92,3], [44,3], [54,3], [66,3], [22,3], [27,3]]
Y = [[2,6], [8,6], [6,4], [18,5], [44,3], [266,3], [66,3], [184,3], [88,3], [108,3], [132,3], [44,3], [54,3]]
Y = np.array(Y, dtype='float')


X = np.array(X, dtype='float')

gradient_descent(X, Y, learning_rate=0.0001)

Converged!


array([[6.66005637e-03],
       [1.99983582e+00],
       [1.73784921e-03]])

In [308]:
def feature_scaling(X):
    """Uses mean normalization to scale the input."""
    
    # Calculate the mean of each feature.
    X_mean = np.mean(X, axis=0).reshape((-1,1))
    X_std = np.std(X, axis=0).reshape((-1,1))

    # If the standard deviation is 0 then we encounter problems.
    if X_std.min(axis=0)[0] == 0:
        return None
    
    return (X - X_mean.T) / X_std.T 

feature_scaling(X)

array([[-1.02046289,  3.27805034],
       [-0.94038632,  0.84292723],
       [-0.96707851, -0.37463432],
       [-0.80692538, -0.37463432],
       [-0.45992694, -0.37463432],
       [ 2.50290596, -0.37463432],
       [-0.16631287, -0.37463432],
       [ 1.40852624, -0.37463432],
       [ 0.12730121, -0.37463432],
       [ 0.39422309, -0.37463432],
       [ 0.71452935, -0.37463432],
       [-0.45992694, -0.37463432],
       [-0.32646599, -0.37463432]])